<a href="https://colab.research.google.com/github/MarynaZinchenko888/python_for_ds_tasks/blob/main/Zinchenko_M__%22HW_%D0%90%D0%BD%D0%B0%D0%BB%D1%96%D0%B7_%D0%90_%D0%92_%D1%82%D0%B5%D1%81%D1%82%D1%96%D0%B2_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency
from statsmodels.stats.proportion import proportion_effectsize, proportions_ztest, proportion_confint
from statsmodels.stats.power import NormalIndPower

data_path = "/content/cookie_cats.csv"
df = pd.read_csv(data_path)

print(df.shape)
df.head()

(90189, 5)


,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [40]:
# Дані з завдання
p_base = 0.19        # базове утримання (контрольна група)
p_min_effect = 0.01  # мінімально відчутний ефект (1%)
alpha = 0.05         # рівень значущості
power = 0.8          # бажана потужність тесту

# Розрахунок розміру вибірки для двох пропорцій
effect_size = (p_base + p_min_effect - p_base) / np.sqrt(p_base * (1 - p_base))
analysis = NormalIndPower()
n_per_group = analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha, alternative='larger')

# Загальна кількість користувачів
n_total = n_per_group * 2

print("\n🔹 Планування A/B-тесту (Cookie Cats)")
print(f"Розмір вибірки на групу ≈ {int(np.ceil(n_per_group))} користувачів")
print(f"Загальна кількість користувачів ≈ {int(np.ceil(n_total))} користувачів")


🔹 Планування A/B-тесту (Cookie Cats)
Розмір вибірки на групу ≈ 19030 користувачів
Загальна кількість користувачів ≈ 38060 користувачів


2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [41]:
data_path = "cookie_cats.csv"
df = pd.read_csv(data_path)

# Середнє утримання через 7 днів по версіях
retention_means = df.groupby("version")["retention_7"].mean()

print("\n🔹 Середнє утримання на 7 день по версіях")
print(retention_means)

# Формулювання гіпотез
print("\nГіпотези:")
print("H0: Утримання на 7 день однакове у обох версіях гри (gate_30 = gate_40)")
print("H1: Утримання на 7 день відрізняється між версіями гри (gate_40 > gate_30), тобто зміщення воріт покращує утримання")


🔹 Середнє утримання на 7 день по версіях
version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64

Гіпотези:
H0: Утримання на 7 день однакове у обох версіях гри (gate_30 = gate_40)
H1: Утримання на 7 день відрізняється між версіями гри (gate_40 > gate_30), тобто зміщення воріт покращує утримання


3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [42]:
# Вибір груп
control = df[df["version"] == "gate_30"]["retention_7"]
treatment = df[df["version"] == "gate_40"]["retention_7"]

# Кількість успіхів (гравці, які повернулись) і розмір групи
successes = np.array([control.sum(), treatment.sum()])
nobs = np.array([control.count(), treatment.count()])

# Z-тест для двох пропорцій (односторонній: treatment > control)
z_stat, p_value = proportions_ztest(successes, nobs, alternative='larger')

# Довірчі інтервали 95% для кожної групи
ci_control = proportion_confint(count=successes[0], nobs=nobs[0], alpha=0.05, method='normal')
ci_treatment = proportion_confint(count=successes[1], nobs=nobs[1], alpha=0.05, method='normal')

In [43]:
print("\n🔹 Результати Z-тесту для retention_7")
print(f"z statistic: {z_stat:.3f}")
print(f"p-value: {p_value:.6f}")
print(f"Довірчий інтервал 95% для групи control (gate_30): [{ci_control[0]:.3f}, {ci_control[1]:.3f}]")
print(f"Довірчий інтервал 95% для групи treatment (gate_40): [{ci_treatment[0]:.3f}, {ci_treatment[1]:.3f}]")


🔹 Результати Z-тесту для retention_7
z statistic: 3.164
p-value: 0.000777
Довірчий інтервал 95% для групи control (gate_30): [0.187, 0.194]
Довірчий інтервал 95% для групи treatment (gate_40): [0.178, 0.186]


In [44]:
print("\nВисновки:")
if p_value < 0.05:
    print("- Статистично значуща різниця між версіями гри є (p < 0.05).")
else:
    print("- Статистично значущої різниці між версіями гри немає (p ≥ 0.05).")

if ci_control[1] < ci_treatment[0] or ci_treatment[1] < ci_control[0]:
    print("- Довірчі інтервали не перетинаються, що підтверджує відмінність утримання між групами.")
else:
    print("- Довірчі інтервали перетинаються, що свідчить про можливу схожість утримання між групами.")


Висновки:
- Статистично значуща різниця між версіями гри є (p < 0.05).
- Довірчі інтервали не перетинаються, що підтверджує відмінність утримання між групами.


4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


In [45]:
# Таблиця спостережень (контингентна таблиця)
contingency_table = pd.crosstab(df["version"], df["retention_7"])

print("🔹 Контингентна таблиця:\n", contingency_table)

# Гіпотези:
# H0: Утримання на 7 день однакове у обох версіях гри (gate_30 = gate_40)
# H1: Утримання на 7 день відрізняється між версіями гри (gate_40 > gate_30), тобто зміщення воріт покращує утримання

chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)


alpha = 0.05       #рівень значущості

print(f"\np-value: {p_value:.6f}")

🔹 Контингентна таблиця:
 retention_7  False  True 
version                  
gate_30      36198   8502
gate_40      37210   8279

p-value: 0.001601


In [46]:
if p_value < alpha:
    print("Висновок: Є статистично значуща залежність між версією гри та утриманням на 7 день, тому H0 відхиляємо.")
else:
    print("Висновок: Немає статистично значущої залежності між версією гри та утриманням на 7 день, тому H0 не відхиляємо.")

Висновок: Є статистично значуща залежність між версією гри та утриманням на 7 день, тому H0 відхиляємо.
